In [ ]:
import pandas as pd
from pandas.core.common import random_state
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedShuffleSplit
from sklearn.metrics import mean_squared_error, mean_squared_log_error

import numpy as np

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/projects/Predict_House_prices/train.csv')

In [ ]:
df_test= pd.read_csv('/content/drive/MyDrive/projects/Predict_House_prices/test.csv')

### Analise train df

In [ ]:
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [ ]:
object_features = ['MSZoning', 'Street', 'LotShape' ,'LandContour' ,'Utilities',	'LotConfig',	'LandSlope',	'Neighborhood',	
                   'Condition1',	'Condition2'	,'BldgType'	,'HouseStyle'	,'RoofStyle',	'RoofMatl',	'Exterior1st',
                   'Exterior2nd',	'MasVnrType'	,'ExterQual'	,'ExterCond',	'Foundation',	'BsmtQual',	'BsmtCond',	'BsmtExposure',
                   'BsmtFinType1', 'BsmtFinType2' ,'Heating'	,'HeatingQC'	,'CentralAir',	'Electrical', 'KitchenQual', 
                   'Functional' ,'GarageType' ,'GarageFinish' ,'GarageQual',	'GarageCond',	'PavedDrive' ,'SaleType',	'SaleCondition']

In [ ]:
# Encoding
le = LabelEncoder()

for i in object_features:
  df_train[i] = le.fit_transform(df_train[[i]].values.ravel())

In [ ]:
df_train = df_train.drop(['Alley', 'PoolQC', 'Fence', 'MiscFeature', 'FireplaceQu'], axis=1)

In [ ]:
df_train.isna().mean().sort_values(ascending=False).head()

LotFrontage    0.177397
GarageYrBlt    0.055479
MasVnrArea     0.005479
Id             0.000000
HalfBath       0.000000
dtype: float64

In [ ]:
df_train['LotFrontage'] = df_train['LotFrontage'].fillna(df_train['LotFrontage'].mean())
df_train['GarageYrBlt'] = df_train['GarageYrBlt'].fillna(df_train['GarageYrBlt'].mean())
df_train['MasVnrArea'] = df_train['MasVnrArea'].fillna(df_train['MasVnrArea'].mean())

### Predict

In [ ]:
features = df_train.drop(['SalePrice', 'Id'], axis=1)
target = df_train['SalePrice'] 

In [ ]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [ ]:
model = RandomForestRegressor(random_state=12345, n_estimators=50, max_depth=10)

model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)

result = mean_squared_error(target_valid, predictions_valid) ** 0.5 #  RMSE metric
result = mean_squared_log_error(target_valid, predictions_valid) ** 0.5 # RMSLE metric
print("RMSLE наилучшей модели на валидационной выборке:", result)

RMSLE наилучшей модели на валидационной выборке: 0.14612242500179456


In [ ]:
%%time


rf = RandomForestRegressor(random_state=123)

params = {'n_estimators': [150, 300],
          'max_depth': [15, 25, 35],
          'min_samples_leaf' : [2],
          'max_features':['auto'],
          'min_samples_split': [2, 5],
          }

cv_rfc_model = GridSearchCV(estimator=rf, param_grid=params, n_jobs=-1, cv=10)

cv_rfc_model.fit(features_train, target_train)

In [ ]:
cv_rfc_model.best_params_

In [ ]:
rfc_best_model = RandomForestRegressor(max_depth=25, min_samples_leaf=2, min_samples_split=4, n_estimators=150, random_state=123, criterion='squared_error')
rfc_best_model.fit(features_train, target_train)

In [ ]:
predictions_valid = rfc_best_model.predict(features_valid)
mean_squared_log_error(target_valid, predictions_valid) ** 0.5

In [ ]:
# RMSLE наилучшей модели на валидационной выборке: 0.14612242500179456

In [ ]:
from sklearn.linear_model import Lasso

Lasso_reg =Lasso()
parameters= {'alpha':[x for x in [0.0005,0.001,0.01,0.1,1]]}

Lasso_reg=GridSearchCV(Lasso_reg, param_grid=parameters)
Lasso_reg.fit(features_train, target_train)
print("The best value of Alpha is: ",Lasso_reg.best_params_,Lasso_reg.best_score_)

In [ ]:
Lasso_reg =Lasso(alpha=0.7409027492746422)
Lasso_reg.fit(features_train, target_train)
y_pred_train=Lasso_reg.predict(features_train)
y_pred_test=Lasso_reg.predict(features_valid)

print('Root Mean Square Error train = ' + str(np.sqrt(mean_squared_log_error(target_train, y_pred_train))))
print('Root Mean Square Error test = ' + str(np.sqrt(mean_squared_log_error(target_valid, y_pred_test))))

Root Mean Square Error train = 0.15343738410675423
Root Mean Square Error test = 0.16674234189504597


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

ridge=Ridge()
parameters= {'alpha':[x for x in [0.1,0.2,0.4,0.5,0.7,0.8,1]]}

ridge_reg=GridSearchCV(ridge, param_grid=parameters)
ridge_reg.fit(features_train, target_train)
print("The best value of Alpha is: ",ridge_reg.best_params_)

The best value of Alpha is:  {'alpha': 1}


In [ ]:
ridge_mod=Ridge(alpha=1)
ridge_mod.fit(features_train, target_train)
y_pred_train=ridge_mod.predict(features_train)
y_pred_test=ridge_mod.predict(features_valid)

print('Root Mean Square Error train = ' + str(np.sqrt(mean_squared_error(target_train, y_pred_train))))
print('Root Mean Square Error test = ' + str(np.sqrt(mean_squared_error(target_valid, y_pred_test)))) 
print('Root Mean Square Error train = ' + str(np.sqrt(mean_squared_log_error(target_train, y_pred_train))))
print('Root Mean Square Error test = ' + str(np.sqrt(mean_squared_log_error(target_valid, y_pred_test))))

Root Mean Square Error train = 31105.651292869043
Root Mean Square Error test = 31565.19410954969
Root Mean Square Error train = 0.1536530977948279
Root Mean Square Error test = 0.16676808628149087


In [ ]:
import xgboost as xgb
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt 

In [ ]:

rfc_best_model = xgb.XGBRegressor(verbosity=0, n_estimators=150)
print(rfc_best_model)

rfc_best_model.fit(features_train, target_train)
 
score = rfc_best_model.score(features_train, target_train)   
print("Training score: ", score) 

XGBRegressor(n_estimators=150, verbosity=0)
Training score:  0.9750291129494205


In [ ]:
# - cross validataion 
scores = cross_val_score(xgbr, features_train, target_train, cv=5)
print("Mean cross-validation score: %.2f" % scores.mean())

kfold = KFold(n_splits=10, shuffle=True)
kf_cv_scores = cross_val_score(xgbr, features_train, target_train, cv=kfold )
print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

Mean cross-validation score: 0.87
K-fold CV average score: 0.89


In [ ]:
ypred = rfc_best_model.predict(features_valid)
mse = mean_squared_log_error(target_valid, ypred) ** 0.5
mse

0.13832249574055386

In [ ]:
%%time


rf = xgb.XGBRegressor(verbosity=0, random_state=123)

params = {'n_estimators': [150, 300, 100, 50, 250, 350, 400]}

cv_rfc_model = GridSearchCV(estimator=rf, param_grid=params, n_jobs=10, cv=10)

cv_rfc_model.fit(features_train, target_train)

CPU times: user 763 ms, sys: 36.1 ms, total: 799 ms
Wall time: 26.4 s


In [ ]:
cv_rfc_model.best_params_

{'n_estimators': 150}

### Analise test df

In [ ]:
# Encoding
le = LabelEncoder()

for i in object_features:
  df_test[i] = le.fit_transform(df_test[i].values.ravel())

In [ ]:
df_test = df_test.drop(['Alley', 'PoolQC', 'Fence', 'MiscFeature', 'FireplaceQu'], axis=1)

In [ ]:
df_test.isna().mean().sort_values(ascending=False).head()

In [ ]:
isna_features = ['LotFrontage', 'GarageYrBlt', 'MasVnrArea', 'BsmtHalfBath', 'BsmtFullBath', 'GarageArea', 'BsmtFinSF1', 'GarageCars', 'TotalBsmtSF', 'BsmtUnfSF', 'BsmtFinSF2']

In [ ]:
for i in isna_features:
  df_test[i] = df_test[i].fillna(df_test[i].mean())

In [ ]:
features_test = df_test.drop('Id', axis=1)

### Predict test

In [ ]:
rfc_best_model.predict(features_test)

array([132854.2 , 159219.84, 177633.33, ..., 162789.47, 121247.34,
       232428.62], dtype=float32)

In [ ]:
df_test['SalePrice'] = list(rfc_best_model.predict(features_test))

In [ ]:
df_test = df_test[['Id', 'SalePrice']]

In [ ]:
df_test.head()

,Id,SalePrice
0,1461,132854.203125
1,1462,159219.843750
2,1463,177633.328125
3,1464,185869.562500
4,1465,189670.234375


In [ ]:
# df_test.to_csv('/content/drive/MyDrive/df_submission_house_v4.csv', index=False)

In [ ]:
# RMSLE 1 score: 0.15319 (my in collab 0.15118550781910392)

# rfc_best_model = RandomForestRegressor(max_depth=25, min_samples_leaf=2, min_samples_split=4, n_estimators=150, random_state=123)
# RMSLE 2 score: 0.15019 (my in collab 0.1440005909127942)

# rfc_best_model = RandomForestRegressor(max_depth=15, min_samples_leaf=2, min_samples_split=2, n_estimators=300, random_state=123)
# RMSLE 3 score: 0.15005 (my in collab 0.14430422042703717)

# XGboost
# RMSLE 4 score: 0.13858